In [ ]:
from googlesearch import search
import json
import requests
from bs4 import BeautifulSoup
import re
import html
import pandas as pd

In [ ]:
# load json data from operaBookmarksExtractor.ipynb
with open('Data/games.json', 'r') as f:
    games_dict = json.load(f)
	
# find all PSNProfiles trophy list urls for each game (there can be many urls as each game could be released more than once on different system)
psnp_urls = []
for g in games_dict['title']:
	print("Getting URLs for", g)
	query = g + " PSNProfiles"
	searchResults = search(query, tld='co.in', num=8, pause=3, stop=8)
	urls = []
	for j in searchResults:
		if ("psnprofiles.com/trophies/" in j) and (j.count('/') == 4) and ('?' not in j):
			urls.append(j)
	psnp_urls.append(urls)
games_dict["psnprofilesTrophyList"] = psnp_urls

with open('Data/games_with_urls.json', 'w', encoding='utf-8') as f:
    json.dump(games_dict, f)


In [ ]:
with open('Data/games_with_urls.json', 'r') as f:
    games_dict = json.load(f)

# get platinum percents, platforms and potential edition for every game
platinum_percents = []
platinum_platforms = []
platinum_editions = []
platinum_titles = []
for i in range(len(games_dict['title'])):
    platsForGame = []
    platformsForGame = []
    editionsForGame = []
    titlesForGame = []
    print("Getting platinum infos for", games_dict['title'][i])
    for idx, u in enumerate(games_dict["psnprofilesTrophyList"][i]):
        proc = ''
        response = requests.get("http://webcache.googleusercontent.com/search?q=cache:" + u + "&vwsrc=1")
        htmlCode = html.unescape(response.text)
        soup = BeautifulSoup(htmlCode, 'html.parser')

        title = soup.find('div', class_='grow')
        t = title.text.replace('\n', '').replace('Games<', '')
        stats = soup.find('div', class_='stats flex')
        platform = soup.find('div', class_='platforms')
        edition = soup.find('th', class_='center', colspan_='2')

        for s in stats.find_all('span', class_='stat grow'):
            if s.find('span').text == 'Platinum Achievers':
                proc = s.text
        if proc == '':
            games_dict["psnprofilesTrophyList"][i].remove(u)
            continue
        match = re.search(r'\d+\.\d+', proc).group()

        with open('Data/GamesHTML/' + t + '_' + str(idx) + '_' + platform.text + '.html', 'w', encoding='utf-8') as f:
            f.write(htmlCode)
        
        platsForGame.append(float(match))
        platformsForGame.append(platform.text)
        titlesForGame.append(t)
        if edition is None:
            editionsForGame.append(None)
        else:
            editionsForGame.append(edition.text)
        
    platinum_percents.append(platsForGame)
    platinum_platforms.append(platformsForGame)
    platinum_editions.append(editionsForGame)
    platinum_titles.append(titlesForGame)
    
games_dict['platinumTitles'] = platinum_titles
games_dict['platinumPlatforms'] = platinum_platforms
games_dict['platinumPercents'] = platinum_percents
games_dict['platinumEditions'] = platinum_editions

In [ ]:
# save games dictionary with urls and platinum percents to json
with open("Data/games_with_platinum_info.json", 'w') as f:
    json.dump(games_dict, f)

In [ ]:
# visualize dictionary as pandas' DataFrame and save it to .csv
df = pd.DataFrame(games_dict)
df.to_csv("Data/games_with_platinum_info.csv")
print(df)